# Compute the Tropical Nights Climate Index

This notebook computes the **Tropical Nights index**: starting from the daily minimum temperature TN,
the Tropical Nights index is the number of days where $TN > T$ (T is  a reference temperature, e.g. 20°C)

As first step, let's connect to the server instance

In [ ]:
from PyOphidia import cube
cube.Cube.setclient(read_env=True)

Import input NetCDF data set (with minimum temperature in °K)

In [ ]:
from os.path import expanduser
home = expanduser("~")
mintemp = cube.Cube.importnc2(src_path=home+"/data/CMIP6/ScenarioMIP/CMCC/CMCC-CM2-SR5/ssp585/r1i1p1f1/Amon/tas/gn/v20200622/tas_Amon_CMCC-CM2-SR5_ssp585_r1i1p1f1_gn_201501-210012.nc",
    measure='tas',
    import_metadata='yes',
    imp_dim='time',
    imp_concept_level='d', vocabulary='CF',hierarchy='oph_base|oph_base|oph_time',
    ncores=1,
    nfrag=1,
    description='Min Temps'
    )

Identify the tropical nights: $\{day \mid TN(day) > 293.15\}$  with apply (we are basically creating a mask)

In [ ]:
tropicalnights = mintemp.apply(
    query="oph_predicate('OPH_FLOAT','OPH_INT',measure,'x-293.15','>0','1','0')"
)

Count the number of yearly tropical nights

In [ ]:
count = tropicalnights.reduce2(
    operation='sum',
    dim='time',
    concept_level='y',
)

Subset on the first year

In [ ]:
firstyear = count.subset(subset_filter=1, subset_dims='time')

Plot the indicator on a map

In [ ]:
%matplotlib inline
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.util import add_cyclic_point
import numpy as np
import warnings
warnings.filterwarnings("ignore")

fig = plt.figure(figsize=(15, 6), dpi=100)

#Add Geo axes to the figure with the specified projection (PlateCarree)
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)

#Draw coastline and gridlines
ax.coastlines()

gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False

data = firstyear.export_array(show_time='yes')
lat = data['dimension'][0]['values'][ : ]
lon = data['dimension'][1]['values'][ : ]
var = data['measure'][0]['values'][ : ]
var = np.reshape(var, (len(lat), len(lon)))

#Wraparound points in longitude
var_cyclic, lon_cyclic = add_cyclic_point(var, coord=np.asarray(lon))
x, y = np.meshgrid(lon_cyclic,lat)

#Define color levels for color bar
levStep = (np.nanmax(var)-np.nanmin(var))/20
clevs = np.arange(np.nanmin(var),np.nanmax(var)+levStep,levStep)

#Set filled contour plot
cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.Oranges)
plt.colorbar(cnplot,ax=ax)

ax.set_aspect('auto', adjustable=None)
plt.title('Tropical Nights (year 2015)')
plt.show()

To clear your workspace before running other notebooks

In [ ]:
cube.Cube.deletecontainer(container='tas_Amon_CMCC-CM2-SR5_ssp585_r1i1p1f1_gn_201501-210012.nc',force='yes')